## 時系列モデルの種類
- AR（自己回帰モデル）　　現在の値は、過去の値から決まる
- MA（移動平均モデル）　　現在の値は、過去の「実績と予測のズレ」から決まる
- ARMA（自己回帰 移動平均モデル） ...AR + MA
- ARIMA（自己回帰 和分 移動平均モデル）  ...ARMA + I(時間帯によって平均値はブレる)
- SARIMA（季節 自己回帰 和分 移動平均モデル） ...S(データに周期性がある) + ARIMA
- SARIMAX ...SARIMA + 説明変数

In [ ]:
#いろいろインポート
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt

#データ読み込み
df = pd.read_csv("3-3_TS_sample2.csv",index_col="年月日",encoding="shift_jis",parse_dates=True)

### AR,MA,ARMA

In [ ]:
#statsmodelsのバージョンは0.8.0以降にしましょう 
#(コマンド)conda install -c taugspurger statsmodels=0.8.0
import statsmodels.api as sm 

y = df["平均気温(℃)"]
arma_model = sm.tsa.ARMA(y, order=(3, 1),freq="D")
result = arma_model.fit()

result.summary()

In [ ]:
# ARMAモデルの最適パラメータ推定
sm.tsa.arma_order_select_ic(df["平均気温(℃)"],ic='aic')

### ARIMA、SARIMA、SARIMAX

In [ ]:
#　月ごとに集計したデータも準備しておく
df_month = df["2010":"2018"].resample("M").mean()
df_month_all = df["2010":"2019"].resample("M").mean()

df_month_pred = df["2019"].resample("M").mean()

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

y = df_month["平均気温(℃)"]
X = df_month[["降水量の合計(mm)","日照時間(時間)"]]
sarima_model = sm.tsa.SARIMAX(y,X, order=(1,1,1),seasonal_order=(0,1,1,12))
result = sarima_model.fit()

result.summary()

### グリッドサーチ

In [ ]:
import itertools

p = q = range(0, 3)
sp = sd = sq = range(0, 2)

#p,q,sp,sd,sqの組み合わせのリストを作成する。和分=1,周期=12は固定
pdq = [(x[0], 1, x[1]) for x in list(itertools.product(p, q))]
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(sp, sd, sq))]

In [ ]:
import warnings
warnings.filterwarnings("ignore")

best_result = [0, 0, 10000000]
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = SARIMAX(y, order = param, seasonal_order = param_seasonal)
            results = mod.fit()
            print('ARIMAパラメータ{}, 季節変動パラメータ{} - AIC: {}'.format(param, param_seasonal, results.aic))
            
            if results.aic < best_result[2]:
                best_result = [param, param_seasonal, results.aic]
        except:
            continue

print('AIC最小のモデル：', best_result)

### ホワイトノイズの確認

In [ ]:
plt.rc("figure",figsize=(20,10))
result.plot_diagnostics(lags=12);

### 波形の予測

In [ ]:
bestPred = result.predict('2018-12', '2019-12',exog=df_month_pred[["降水量の合計(mm)","日照時間(時間)"]])
plt.plot(df_month_all["平均気温(℃)"])
plt.plot(bestPred, "r")

In [ ]:
bestPred